### This notebook was created to help users test and debug their own custom nodes. 
See [here](https://optinist.readthedocs.io/en/latest/specifications/add_algorithm.html) in the documentation for details. 

# Setup:
1. Create environment:
    In terminal, run:
    
    `conda env create -n custom_node_test -f studio/app/optinist/wrappers/optinist/conda/optinist.yaml`

    `conda activate custom_node_test`

2. Install some additional packages:
    We have included all of the packages for CaImAn, Suite2p and LCCD here, for easy combination of those 
    packages with your custom node. See the other notebooks for examples of how to use those packages. 

   `pip install pynwb imageio ipython jupyter notebook "pydantic<2.0.0" python-dotenv uvicorn xmltodict  bcrypt matplotlib "scikit-learn==1.1.*" plotly`
  - If running in VS code, you may need to restart and/or select the correct environment with "Python: Select Interpreter"

3. Run your notebook

In [3]:
import os
import sys
import uuid

sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('.'))

from studio.app.dir_path import DIRPATH
from studio.app.common.core.logger import AppLogger

logger = AppLogger.get_logger()

# Create input directories based on default saving path
input_dir = os.path.join(DIRPATH.INPUT_DIR, "1")
os.makedirs(input_dir, exist_ok=True)
unique_id = str(uuid.uuid4())[:8]  # Generate 8-char unique ID

# Create output directory 
my_function_id = f"my_func_{unique_id}"
my_output_dir = os.path.join(DIRPATH.OUTPUT_DIR, "1", unique_id, my_function_id)
os.makedirs(my_output_dir, exist_ok=True)

In [4]:
# Import standard libraries
import numpy as np
import pandas as pd

# Import OptiNiSt core data modules
# Examples:
from studio.app.optinist.core.nwb.nwb import NWBDATASET
from studio.app.common.core.experiment.experiment import ExptOutputPathIds
from studio.app.common.dataclass import CsvData, ImageData, TimeSeriesData, HTMLData
from studio.app.optinist.dataclass import FluoData, BehaviorData, CaimanCnmfData, IscellData, LccdData, NWBFile, RoiData, SpikingActivityData, Suite2pData

# Import ROI detection modules
# Examples:
# from studio.app.optinist.wrappers.suite2p import file_convert, registration, roi
# from studio.app.optinist.wrappers.caiman import motion_correction, cnmf
# from studio.app.optinist.wrappers.lccd import lccd_detection

# Import OptiNiSt analysis modules
# Examples:
# from studio.app.optinist.wrappers.optinist.basic_neural_analysis.eta import ETA
# from studio.app.optinist.wrappers.optinist.my_custom_node import MyCustomNode

# Import OptiNiSt visualization modules
# Examples:
# from studio.app.common.dataclass import BarData, HeatMapData, HistogramData, ScatterData, LineData, PieData, PolarData, ScatterData
from studio.app.common.dataclass import BarData, LineData

# Import visualization modules
# from caiman.utils.visualization import local_correlations
# from plotly.subplots import make_subplots
# import plotly.graph_objects as go
# import plotly.express as px

In [5]:
# Set parameters. These should be saved in the folder XXXX
my_params = {}

In [6]:
def my_analysis_function(
    # Required inputs
    neural_data: FluoData,  # Fluorescence data from previous processing
    output_dir: str,        # Directory to save output files
    # Optional inputs
    iscell: IscellData = None,  # Cell classification data if needed
    params: dict = None,        # Additional parameters to customize processing
    **kwargs                    # Catch-all for additional arguments
) -> dict:  # Function returns a dictionary containing all outputs
    """Example template for creating analysis functions.
    
    This function shows the basic structure for creating analysis functions
    that work with the pipeline, including proper input handling, NWB file
    creation, and return format.
    
    Args:
        neural_data: Fluorescence data from previous processing steps
        output_dir: Directory where output files should be saved
        iscell: Optional cell classification data
        params: Optional dictionary of parameters to customize processing
        **kwargs: Additional keyword arguments
        
    Returns:
        dict: Dictionary containing all output data and metadata
    """
    
    # 1. Set up logging if needed
    logger.info("Starting my_analysis_function")
    
    # 2. Get additional data from kwargs if needed
    nwbfile = kwargs.get("nwbfile", {})
    
    # 3. Set default parameters and update with user params
    default_params = {
        "window_size": 10,
        "threshold": 0.5
    }
    if params is not None:
        default_params.update(params)
    
    # 4. Main analysis code goes here
    my_analysis = np.random.rand(10), np.random.rand(10)  # Example data

    my_classification = my_analysis > default_params["threshold"]
    
    # 5. Prepare NWB file structure
    # Create a new NWB file dictionary or update existing one
    nwb_output = {}
    
    # Add ROIs if your analysis creates them
    nwb_output[NWBDATASET.ROI] = {} # List of ROI dictionaries
    
    # Add processing results
    nwb_output[NWBDATASET.POSTPROCESS] = {
        my_analysis: {
            "analysis_result": my_analysis[0]  # Your analysis outputs
        }
    }
    
    # Add column data (like classifications)
    nwb_output[NWBDATASET.COLUMN] = {
        my_classification: {
            "name": "my_classification",
            "description": "Description of the classification",
            "data": my_analysis[1]
        }
    }
    
    # 6. Prepare return dictionary
    # This should contain all outputs and processed data
    info = {
        # Analysis results in appropriate data containers
        "my_bar_data": BarData(my_analysis[0], output_dir=output_dir, file_name="bar_data"),
        "my_line_data": LineData(my_analysis[1], output_dir=output_dir, file_name="line_data"),
        
        # Add NWB file data
        "nwbfile": nwb_output
    }
    
    return info